In [1]:
import pandas as pd

In [3]:
df_hash = pd.read_csv("results_GA_ADJ_MEM.csv")

df_noHash = pd.read_csv("results_GA_ADJ_MEM_noHash.csv")

print(df_hash.columns.tolist())
print(df_noHash.columns.tolist())

['instancia', 'a', 'b', 'c', 'v', 'N', 'M', 'mean_chi', 'se_chi', 'mean_time', 'se_time', 'peak_mem_mb', 'mean_mem_mb']
['instancia', 'a', 'b', 'c', 'v', 'N', 'M', 'mean_chi', 'se_chi', 'mean_time', 'se_time', 'peak_mem_mb', 'mean_mem_mb']


In [5]:
# df para comparação de resultados e obtenção de estatísticas sobre eles
df_comp_MEM = pd.merge(df_hash, df_noHash, on="instancia", suffixes=('_hash', '_noHash'))

# analisar se há alguma diferença de resultado em termos de qualidade das colorações obtidas
df_comp_MEM['diff_chi'] = df_hash['mean_chi'] - df_noHash['mean_chi']

# como o que queremos verificar é se o uso da tabela Hash pode gerar redução no tempo de
# execução em troca de uso razoável de memória extra, as métricas mais importantes serão o tempo
# (com hash deve ser menor), o pico de uso de memória e a memória média (devem ser razoáveis)
# (partimos da premissa de que os resultados em qualidade de colorações serão similares ou idênticos,
# uma vez que o método é praticamente o mesmo, com alteração somente na avaliação de fitness)
df_comp_MEM['diff_tempo'] = df_hash['mean_time'] - df_noHash['mean_time']

# Percentual de alteração no tempo de execução
# se positivo: o uso de hash demorou mais (pior performance)
# se negativo: o uso de hash foi mais rápido (melhor performance) - é o que esperamos
df_comp_MEM['time_change_pct'] = ((df_comp_MEM['mean_time_hash'] - df_comp_MEM['mean_time_noHash']) / df_comp_MEM['mean_time_noHash']) * 100

df_comp_MEM['speedup'] = df_comp_MEM['mean_time_noHash'] / df_comp_MEM['mean_time_hash']

# diferenças em uso de memória (pico e médias)
df_comp_MEM['diff_mean_MEM'] = df_hash['mean_mem_mb'] - df_noHash['mean_mem_mb']

df_comp_MEM['diff_pico_MEM'] = df_hash['peak_mem_mb'] - df_noHash['peak_mem_mb']

# aumento percentual no uso de memória
df_comp_MEM['MEM_increase_pct'] = ((df_comp_MEM['mean_mem_mb_hash'] - df_comp_MEM['mean_mem_mb_noHash']) / df_comp_MEM['mean_mem_mb_noHash']) * 100

df_comp_MEM['Peak_MEM_increase_pct'] = ((df_comp_MEM['peak_mem_mb_hash'] - df_comp_MEM['peak_mem_mb_noHash']) / df_comp_MEM['peak_mem_mb_noHash']) * 100

In [ ]:
# qualidade em cores
df_comp_MEM['diff_chi'] = df_comp_MEM['mean_chi_hash'] - df_comp_MEM['mean_chi_noHash']

# performance em tempo
df_comp_MEM['diff_tempo'] = df_comp_MEM['mean_time_hash'] - df_comp_MEM['mean_time_noHash']
df_comp_MEM['pct_change_time'] = ((df_comp_MEM['mean_time_hash'] - df_comp_MEM['mean_time_noHash']) / df_comp_MEM['mean_time_noHash']) * 100
df_comp_MEM['speedup'] = df_comp_MEM['mean_time_noHash'] / df_comp_MEM['mean_time_hash']
# slowdown (contrário do speedup, pode ser melhor para visualização)
df_comp_MEM['slowdown_tempo'] = df_comp_MEM['mean_time_hash'] / df_comp_MEM['mean_time_noHash']

# consumo de memória
df_comp_MEM['diff_mean_MEM'] = df_comp_MEM['mean_mem_mb_hash'] - df_comp_MEM['mean_mem_mb_noHash']
df_comp_MEM['pct_increase_mem'] = ((df_comp_MEM['mean_mem_mb_hash'] - df_comp_MEM['mean_mem_mb_noHash']) / df_comp_MEM['mean_mem_mb_noHash']) * 100
df_comp_MEM['pct_increase_peak_mem'] = ((df_comp_MEM['peak_mem_mb_hash'] - df_comp_MEM['peak_mem_mb_noHash']) / df_comp_MEM['peak_mem_mb_noHash']) * 100


# geração de estatísticas de interesse no df construído
stats_qualidade = df_comp_MEM[['mean_chi_hash', 'mean_chi_noHash', 'diff_chi']].describe()
stats_tempo = df_comp_MEM[['mean_time_hash', 'mean_time_noHash', 'diff_tempo', 'pct_change_time', 'speedup']].describe()
stats_memoria = df_comp_MEM[['mean_mem_mb_hash', 'mean_mem_mb_noHash', 'diff_mean_MEM', 'pct_increase_mem']].describe()

print("--- Estatísticas de Qualidade (Cores) ---")
print(stats_qualidade)

print("\n--- Estatísticas de Performance (Tempo) ---")
print(stats_tempo)

print("\n--- Estatísticas de Consumo (Memória) ---")
print(stats_memoria)

# resumo de vitórias e eficiência comparativas
mais_rapido = (df_comp_MEM['mean_time_hash'] < df_comp_MEM['mean_time_noHash']).sum()
tempo_igual = (df_comp_MEM['mean_time_hash'] == df_comp_MEM['mean_time_noHash']).sum()
mais_lento = (df_comp_MEM['mean_time_hash'] > df_comp_MEM['mean_time_noHash']).sum()

# casos em que o impacto de memória foi baixo (abaixo de 5%)
mem_extra_minima = (df_comp_MEM['pct_increase_mem'] < 5).sum() 

print(f"\nResumo de Eficiência (Hash vs Sem Hash):")
print(f"Hash mais rápido: {mais_rapido}")
print(f"Tempo equivalente: {tempo_igual}")
print(f"Hash mais lento: {mais_lento}")
print(f"Instâncias com baixo impacto de memória (<5%): {mem_extra_minima}")

--- Estatísticas de Qualidade (Cores) ---
       mean_chi_hash  mean_chi_noHash    diff_chi
count     189.000000       189.000000  189.000000
mean       77.620106        77.571429    0.048677
std        27.453919        27.365521    0.807782
min        12.000000        12.000000   -2.600000
25%        59.800000        59.800000   -0.200000
50%        81.000000        80.600000    0.000000
75%        99.200000        99.200000    0.400000
max       133.800000       133.800000    3.200000

--- Estatísticas de Performance (Tempo) ---
       mean_time_hash  mean_time_noHash  diff_tempo  pct_change_time  \
count      189.000000        189.000000  189.000000       189.000000   
mean        33.690430         34.361619   -0.671189         5.329396   
std         48.879356         52.805209   19.110463        37.302902   
min          0.001330          0.001420 -117.234340       -71.098682   
25%          1.579260          1.373560   -2.921640       -15.996422   
50%         13.907230         1